# Basic download from encode script.

In [1]:
import pandas as pd
import urllib
import json
import requests
import os
host = 'https://www.encodeproject.org'
experiments = "https://www.encodeproject.org/experiments/"

In [2]:
expt = 'ENCSR629EWX' # shRNA knockdown followed by RNA-seq (shRNA RNA-seq)
# experiment_accession \t cell type \t rbp \t rep1_bam \t rep2_bam \t control_accession \t control_rep1_bam \t control_rep2_bam

df = pd.read_table('/home/bay001/projects/encode/permanent_data/metadata.tsv')
def star_or_tophat(link):
    # print("LINK",os.path.basename(link))
    if os.path.basename(link).find('star') > -1:
        return 'STAR'
    if os.path.basename(link).find('tophat') > -1:
        return 'TOPHAT'
    return 'ambiguous'

In [3]:
sample_manifest = '/home/bay001/projects/encode/permanent_data/rna_seq_manifest_TEST.txt' #12-8-2016.txt'
X = df.head(10)
samples = list()
links = list()
controls = list()
error = list()
for h in X['File accession'].dropna():
    url = experiments+h+"/?format=json"
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    if 'code' in data.keys():
        next
    else:
        f = data['replicate']
        expt = f['experiment']['possible_controls'][0].replace('/experiments/','').replace('/','')
        biological_replicate_number = f['biological_replicate_number']
        assembly = data['assembly']
        aligner = star_or_tophat(data['submitted_file_name'])
        # print(expt,biological_replicate_number,assembly,aligner)
        url2 = experiments+expt+"/?format=json"
        response2 = urllib.urlopen(url2)
        
        data2 = json.loads(response2.read())
        # print(data2['assembly'])
        for i in range(0,len(data2['files'])):
            if ((host+data2['files'][i]['href']).endswith('bam') & 
            (data2['files'][i]['output_type'] == u'alignments') &
            (data2['files'][i]['lab'][u'name'] == 'encode-processing-pipeline') &
            (data2['files'][i]['replicate']['biological_replicate_number'] == biological_replicate_number) &
            (star_or_tophat(data2['files'][i]['submitted_file_name'])==aligner)):
                #for key, value in data2['files'][i]['submitted_file_name'].iteritems():
                #for key, value in data2['files'][i]['replicate']['library'].iteritems():
                #    print("key",key)
                #    print("value",value)
                # print(data2['files'][i]['output_type'])
                if(data2['files'][i]['assembly']==assembly):
                    samples.append({'sample':h+".bam",'control':os.path.basename(data2['files'][i]['href'])})
                
        """
        for i in range(0,len(data['files'])):
            if ((host+data['files'][i]['href']).endswith('bam') & 
            (data['files'][i]['output_type'] == u'alignments') &
            (data['files'][i]['lab'][u'name'] == 'encode-processing-pipeline')):
                samples.append({'biological_replicate_number':data['files'][i]['replicate']['biological_replicate_number'],
                                'submitted_expt_file_name':data['files'][i]['submitted_file_name'],
                                'expt_file_name':os.path.basename(data['files'][i]['href']),
                                'assembly':data['files'][i]['assembly'],
                                'expt_url':host+data['files'][i]['href'],
                                'RNASEQ_ControlENC':data['possible_controls'][0]['accession'],
                                'RNASEQ_ENCODEAccID':h,
                                'aligner': star_or_tophat(data['files'][i]['submitted_file_name'])
                               })
                links.append(host+data['files'][i]['href'])
        controls.append(data['possible_controls'][0]['accession'])"""
samples_df = pandas.DataFrame(samples)

samples_df.to_csv(sample_manifest,
                  sep='\t',
                  index=None)

samples_df

NameError: name 'pandas' is not defined

In [100]:
df.ix[0]

File accession                                                                    ENCFF689YFW
File format                                                                               bam
Output type                                                                        alignments
Experiment accession                                                              ENCSR492BKM
Assay                                                     shRNA knockdown followed by RNA-seq
Biosample term id                                                                 EFO:0002067
Biosample term name                                                                      K562
Biosample type                                                         immortalized cell line
Biosample life stage                                                                    adult
Biosample sex                                                                          female
Biosample organism                                          

In [6]:
df['File download URL'][0]

'https://www.encodeproject.org/files/ENCFF689YFW/@@download/ENCFF689YFW.bam'